<h1> Imported Packages</h1>

In [1]:
import cv2
import numpy as np
import os
import random
import re
from itertools import permutations
import time
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

<h1>Dataset Generation</h1>

In [2]:

def rotate_image(image, angle):
    image_center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
    result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
    return result

def skew_image(image,direction):
    
    factor = random.uniform(0.01,0.05)
    
    w = image.shape[1]
    h = image.shape[0]
    
    #initial coords of the 4 corners, ahseb fil google lens taghzel il kantunieri tal karta
    points1 = np.float32([[0, 0], [image.shape[1], 0], [0, image.shape[0]], [image.shape[1], image.shape[0]]])
    
    #jaghzel new anchor points tal corners to skew towards
    if direction == 1:
        points2 = np.float32([
                    [0, 0],
                        [w, 0],
                        [int(w * factor), h],
                        [w - int(w * factor), h]
                        ])
    elif direction == 2:
        points2 = np.float32([
                    [0, 0],
                        [w, int(h * factor)],
                        [0, h],
                        [w, int(h - h * factor)]
                        ])
    elif direction == 3:
        points2 = np.float32([
                    [0, int(h * factor)],
                        [w, 0],
                        [0, int(h - h * factor)],
                        [w, h]
                        ])
    else:
        points2 = np.float32([
                        [int(w * factor), 0],
                        [w - int(w * factor), 0],
                        [0, h],
                        [w, h]
                        ])
    #finds the transformation matrix that transforms the image depending on anchor points   
    transform_matrix = cv2.getPerspectiveTransform(points1, points2)
    
    #warps the perspective of the image based on the above transformation matrix
    return cv2.warpPerspective(image, transform_matrix, (w, h), flags = cv2.INTER_NEAREST)


def noise(image):
    
    
    gauss = np.random.normal(0,0.1**0.2,image.size)
    gauss = gauss.reshape(image.shape[0],image.shape[1],image.shape[2]).astype('uint8')
    return cv2.add(image,gauss)
    
    

for constellation in os.listdir("Constellations"):
    
    print(constellation)
    
    for x in range(0,20):

        new_image = cv2.imread("Constellations/"+constellation,1)

        #contrast
        alpha = random.uniform(1,2)
        
        #brightness
        beta = random.randint(-20,15)
        new_image = cv2.convertScaleAbs(new_image, alpha=alpha, beta=beta)

        new_image = cv2.copyMakeBorder(new_image,new_image.shape[0]//4,new_image.shape[0]//4,new_image.shape[1]//4,new_image.shape[1]//4,cv2.BORDER_CONSTANT,value = 0)


        #new_image = skew_image(new_image, random.randint(1,4))

        angle = random.randint(1,360)
        new_image = rotate_image(new_image,angle)
     
        cv2.imwrite('Dataset/'+constellation.split(".")[0]+str(x)+'.png', new_image) 



Cassiopeia.PNG
Gemini.PNG
Hercules.PNG
Leo.PNG
Libra.PNG
Perseus.PNG
Phoenix.PNG
UrsaMinor.PNG


<h1> Template Database </h1>


In [3]:
def binarizeImg(image,threshold):

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    blurred = cv2.blur(gray,(4,4))
        
    ret,binary = cv2.threshold(blurred,threshold,255,cv2.THRESH_BINARY++cv2.THRESH_OTSU)
   
    return binary




for line_image in os.listdir("Constellations"):
    
    print(line_image)
    
    
    image = cv2.imread("Constellations/"+line_image,1)
    
    
    binarized = binarizeImg(image,150)
    
    cv2.imwrite('TemplateDataset/'+line_image, binarized) 

Cassiopeia.PNG
Gemini.PNG
Hercules.PNG
Leo.PNG
Libra.PNG
Perseus.PNG
Phoenix.PNG
UrsaMinor.PNG


<h1> Pre-Processing Test Image</h1>

In [4]:
def preprocessing(image,threshold):
   
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    blurred = cv2.blur(gray,(4,4))

    ret,binary = cv2.threshold(gray,threshold,255,cv2.THRESH_BINARY)
   
    return binary
    
    


for data_image in os.listdir("Dataset"):
        
    image = cv2.imread("Dataset/"+data_image,1)
    
    binarized = preprocessing(image,150)
    
    cv2.imwrite('ProcessedDataset/'+data_image, binarized) 

<h1> Contour Functions</h1>

In [5]:

# from a binary image, extract all the contours and return the largest contour object
def get_largest_contour(image):

    contours, hierarchy= cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    
    sorted_contours= sorted(contours, key=cv2.contourArea, reverse= True)

    return sorted_contours[0]

    
#returns a sorted list, by area starting with the largest, of the centres of the contours
def findFourLargestStar(binary_image):
    
    contours, hierarchy= cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    
    sorted_contours= sorted(contours, key=cv2.contourArea, reverse= True)

    
    to_draw = binary_image.copy()
    to_draw = cv2.cvtColor(to_draw, cv2.COLOR_GRAY2BGR)
    
    
    #a debugging line used to visualise the four brightest stars that were selected
    '''largest_item= sorted_contours[0]
    second_item = sorted_contours[1]
    third_item= sorted_contours[2]
    fourth_item= sorted_contours[3]        
    
    
    

    cv2.drawContours(to_draw, largest_item, -1, (0,0,255),10)
    cv2.drawContours(to_draw, second_item, -1, (255,0,0),10)
    cv2.drawContours(to_draw, third_item, -1, (0,255,0),10)
    cv2.drawContours(to_draw, fourth_item, -1, (255,255,0),10)
    cv2.waitKey(0)
    cv2.imshow('Largest Object', to_draw)'''
    
    contour_centres = []
    
    for contour in sorted_contours:
        #using image moments to determine the centre of a contour for each contour object
        M = cv2.moments(contour)
        
        if  M["m00"] == 0:
            break
        
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
        
        contour_centres.append( (cX,cY) )
    
    return contour_centres

<h1> Contour Matching</h1>

In [6]:
def contourMatching(destinationStars,sourceStars,testImage,templateImage):
    #initialises the four corresponding points between the two images
    
    pts_source = np.array([sourceStars[0],sourceStars[1],sourceStars[2],sourceStars[3]])
    
    pts_destination = np.array([destinationStars[0],destinationStars[1],destinationStars[2],destinationStars[3]]) 
    
    #find the 3d transformation matrix that maps the first set of points onto the next
    h, status = cv2.findHomography(pts_source, pts_destination)
    
    #perfroms the transformation on the template image
    rotated_template = cv2.warpPerspective(templateImage, h, (testImage.shape[1],testImage.shape[0]))
       
    rot_temp_stars = findFourLargestStar(rotated_template)
   
    largestStar = get_largest_contour(rotated_template)
    
    #calculates the differnce between the largest contours found in each object, if they are the same image, will be 0.
    #therefore a closer image would have a smaller value.
    
    matching = cv2.matchShapes(largestStar,get_largest_contour(testImage),cv2.CONTOURS_MATCH_I1,0.0)
    return matching
    

<h1> Star Positioning</h1>

In [7]:
def starPositions(destinationStars,sourceStars,testImage,templateImage):
    
    pts_source = np.array([sourceStars[0],sourceStars[1],sourceStars[2],sourceStars[3]])
    
    pts_destination = np.array([destinationStars[0],destinationStars[1],destinationStars[2],destinationStars[3]]) 
    
    h, status = cv2.findHomography(pts_source, pts_destination)

    rotated_template = cv2.warpPerspective(templateImage, h, (testImage.shape[1],testImage.shape[0]))
    
    
    rot_temp_stars = findFourLargestStar(rotated_template)
   
    largestStar = rot_temp_stars[0]
    
    
    
    '''    
    to_draw_original = testImage.copy()
    to_draw_original = cv2.cvtColor(to_draw_original, cv2.COLOR_GRAY2BGR)
    
    to_draw_temp = rotated_template.copy()
    to_draw_temp = cv2.cvtColor(to_draw_temp, cv2.COLOR_GRAY2BGR)'''
       
    #print(cX,cY)
    
    confidence = []
    
    max_no_of_stars = min(len(rot_temp_stars),len(destinationStars))
    colours = [(255,0,0),(0,255,0),(0,0,255),(255,255,0),(255,255,255),(255,0,255),(0,255,255)]
    for x in range(0,max_no_of_stars):
        
        
        #old debugging functions that helped determine the correct matching of the stars, these do not work anymore since no 
        #longer using whole contours but rather only the centres.
        
        '''colour = random.randint(50,255)
        cv2.drawContours(to_draw_original, destinationStars[x], -1, (colour,colour,colour),10)
        cv2.drawContours(to_draw_temp, rot_temp_stars[x], -1, (colour,colour,colour),10)
        '''
        
        lit = checkNeighbourhood(rot_temp_stars[x],testImage)
        confidence.append(lit)
    
    norm_confidence = confidence.count(255)/len(confidence)
        
    
    
    final = cv2.hconcat([testImage,rotated_template])
       
    
    #displays the original test image alongside the transformed template
    '''
    cv2.imshow("Rotated Template",final)
    cv2.waitKey(0)'''
   
    return norm_confidence
    
#will check around the neighbourhood of a pixel to see if there is a white pixel, scan range size set by scan.
def checkNeighbourhood(coord,testImage):
    
    
    scan = 5
    
    for x in range(-scan,scan):
        for y in range(-scan,scan):
            
            if coord[1]+y >= testImage.shape[0] or coord[1]+y < 0:
                break
                
            if coord[0]+x >= testImage.shape[1] or coord[0]+x < 0:
                break
            
            if testImage[coord[1]+y,coord[0]+x] == 255:
                return 255
            
    return 0 
    

In [8]:

def permutationsTemplateMatching(destinationStars,sourceStars,testImage,templateImage):
    
    
    pts_source = np.array([sourceStars[0],sourceStars[1],sourceStars[2],sourceStars[3]])
    
    
    permutations_list = list(permutations([0,1,2,3]))
    
    prev_best = 9999
    #loops through all permutations, all possible configurations of the top 4 stars being in first second third or fourth
    for z in range(0,len(permutations_list)):
        first = permutations_list[z][0]
        second = permutations_list[z][1]  
        third = permutations_list[z][2]
        fourth = permutations_list[z][3]
        pts_destination = np.array([destinationStars[first],destinationStars[second],destinationStars[third],destinationStars[fourth]]) 
        h, status = cv2.findHomography(pts_source, pts_destination)

        rotated_template = cv2.warpPerspective(templateImage, h, (testImage.shape[1],testImage.shape[0]))
        
    
        
        rot_temp_stars = findFourLargestStar(rotated_template)
        
        
        #transforms the template at each iteration and performs the contour mtaching to see which produces the least distortion
        largestStar = get_largest_contour(rotated_template)
        matching = cv2.matchShapes(largestStar,get_largest_contour(testImage),cv2.CONTOURS_MATCH_I1,0.0)
        
        
        
        if matching < prev_best:
            prev_best = matching
            best_index = z
        
        
        
        
   
    #obtains the transformed template that produced the least distortion 
    first = permutations_list[best_index][0]
    second = permutations_list[best_index][1]
    third = permutations_list[best_index][2]
    fourth = permutations_list[best_index][3]
    pts_destination = np.array([destinationStars[first],destinationStars[second],destinationStars[third],destinationStars[fourth]]) 
    h, status = cv2.findHomography(pts_source, pts_destination)

    rotated_template = cv2.warpPerspective(templateImage, h, (testImage.shape[1],testImage.shape[0]))

    
    rot_temp_stars = findFourLargestStar(rotated_template)
    
    largestStar = rot_temp_stars[0]
    
    #performs the rest of the Star Positioning function
    
    to_draw_original = testImage.copy()
    to_draw_original = cv2.cvtColor(to_draw_original, cv2.COLOR_GRAY2BGR)
    
    to_draw_temp = rotated_template.copy()
    to_draw_temp = cv2.cvtColor(to_draw_temp, cv2.COLOR_GRAY2BGR)
       
    #print(cX,cY)
    
    confidence = []
    
    max_no_of_stars = min(len(rot_temp_stars),len(destinationStars))
    
    for x in range(0,max_no_of_stars):
        
        '''
        colour = random.randint(1,255)
        cv2.drawContours(to_draw_original, destinationStars[x], -1, (colour,59,colour),10)
        cv2.drawContours(to_draw_temp, rot_temp_stars[x], -1, (colour,59,colour),10)
        '''
        #print(destinationStars[x][0],rot_temp_stars[x][0])
        
        #print(testImage[rot_temp_stars[x][0][0][1],rot_temp_stars[x][0][0][0]])
        #print(testImage[destinationStars[x][0][0][1],destinationStars[x][0][0][0]])
        
        lit = checkNeighbourhood(rot_temp_stars[x],testImage)
        confidence.append(lit)
    
    norm_confidence = confidence.count(255)/len(confidence)
        
    #print(confidence.count(255)/len(confidence))
    
    final = cv2.hconcat([testImage,rotated_template])
    final2 = cv2.hconcat([to_draw_original,to_draw_temp])
    
    
    '''
    cv2.imshow("Rotated Template",final)
    cv2.waitKey(0)
    ''''''
    cv2.imshow("Rotated Template",final2)
    cv2.waitKey(0)'''
    
    return norm_confidence

In [18]:

def drawConstellation(destinationStars,image_name,template_name,permute):
    
    #loads the template image of the predicted template's lines.
    lines = cv2.imread("ConstellationLines/"+template_name,1)
    
    #loads the original test image
    original_image = cv2.imread("Dataset/"+image_name,1)
    
    #only extract RED values, so we only plot the red colours within the template.
    image_copy = lines.copy()
    image_copy[:, :, 0] = 0
    image_copy[:, :, 1] = 0
    
    template = cv2.imread("TemplateDataset/"+template_name,0)

    templateStars = findFourLargestStar(template)
    
    #performs the process of locating the four brightest stars on both images and then transforming the template 
    pts_source = np.array([templateStars[0],templateStars[1],templateStars[2],templateStars[3]])
    
    if permute == False:
        pts_destination = np.array([destinationStars[0],destinationStars[1],destinationStars[2],destinationStars[3]]) 

    else:
        permutations_list = list(permutations([0,1,2,3]))
    
        prev_best = 9999

        for z in range(0,len(permutations_list)):
            first = permutations_list[z][0]
            second = permutations_list[z][1]
            third = permutations_list[z][2]
            fourth = permutations_list[z][3]
            pts_destination = np.array([destinationStars[first],destinationStars[second],destinationStars[third],destinationStars[fourth]]) 
            h, status = cv2.findHomography(pts_source, pts_destination)

            rotated_template = cv2.warpPerspective(template, h, (original_image.shape[1],original_image.shape[0]))
            rot_temp_stars = findFourLargestStar(rotated_template)

            largestStar = rot_temp_stars[0]
            matching = cv2.matchShapes(largestStar,destinationStars[0],cv2.CONTOURS_MATCH_I1,0.0)

            if matching < prev_best:
                prev_best = matching
                best_index = z

        
        
        
        

        first = permutations_list[best_index][0]
        second = permutations_list[best_index][1]
        third = permutations_list[best_index][2]
        fourth = permutations_list[best_index][3]
        pts_destination = np.array([destinationStars[first],destinationStars[second],destinationStars[third],destinationStars[fourth]]) 
        
    h, status = cv2.findHomography(pts_source, pts_destination)

    rotated_lines = cv2.warpPerspective(lines, h, (original_image.shape[1],original_image.shape[0]))
    rotated_lines[:, :, 0] = 0
    rotated_lines[:, :, 1] = 0
    
    #draws the rotated template onto the original image
    result = cv2.addWeighted(original_image,0.7,rotated_lines,0.3,0)
    
    '''cv2.imshow("Connect The Dots",result)
    cv2.waitKey(0)
    '''
    #saves the results 
    cv2.imwrite("Results/"+image_name,result)
    

In [ ]:



#Do Not Touch
correct = 0
total = 0
start = time.time()

predicted = []
actual = []

#Touch

#Choose Method to execute, Contour Matching, Star Positioning or else defaults to Star Brightness Correction
method = "Contour Matching"
#True if you want to draw the results and save them in the results folder, False otherwise
draw = True

#loops through all images in the dataset
for image in os.listdir("ProcessedDataset"):
    
    total+=1
    
    test = cv2.imread("ProcessedDataset/"+image,0)
    #print(test.shape)
    
    testStars = findFourLargestStar(test)
    
    confidences = []
    
    contourMatches = []
    
    #checks it with every available template
    for template in os.listdir("TemplateDataset"):
    
        cassTemplate = cv2.imread("TemplateDataset/"+template,0)

        templateStars = findFourLargestStar(cassTemplate)
        
        if method == "Contour Matching":
            permute = False
            match = contourMatching(testStars,templateStars,test,cassTemplate)
        
            contourMatches.append(match)

        elif method == "Star Positioning":
            permute = False
            confidence = starPositions(testStars,templateStars,test,cassTemplate)
            confidences.append(confidence)
        else:
            permute = True
            confidence = permutationsTemplateMatching(testStars,templateStars,test,cassTemplate)
            confidences.append(confidence)
    
    
    if method == "Contour Matching" :
        smallest = min(contourMatches)
        index = contourMatches.index(smallest)
        
        #regex expression to retrieve the name of the constellation only from the image
        a = re.split('[^a-zA-Z]', image)
        a = a[0].upper()

        b = re.split('[^a-zA-Z]', os.listdir("TemplateDataset")[index])
        b = b[0].upper()

        print("Actual:",image," Matched:",os.listdir("TemplateDataset")[index])

        actual.append(a)
        predicted.append(b)

        if a == b:
            correct +=1 
    else:
        largest = max(confidences)

        index = confidences.index(largest)

        print("Actual:",image," Matched:",os.listdir("TemplateDataset")[index],largest*100,"%")

        a = re.split('[^a-zA-Z]', image)
        a = a[0].upper()

        b = re.split('[^a-zA-Z]', os.listdir("TemplateDataset")[index])
        b = b[0].upper()
        
        actual.append(a)
        predicted.append(b)
        
        if a == b:
            correct +=1 
            
    if draw == True:       
        drawConstellation(testStars,image,os.listdir("TemplateDataset")[index],permute)


#constructs a confusion matrix based on the predicted and actual labels, and then displays it
confusion = confusion_matrix(actual, predicted)

display = ConfusionMatrixDisplay(confusion, ["CASS","GEM","HERC","LEO","LIBR","PERS","PHNX","URSA"])


display.plot()
   
print("Model Accuracy:",correct/total*100)  
print("Time Taken: ",time.time()-start)

Actual: Cassiopeia0.png  Matched: Hercules.PNG
Actual: Cassiopeia1.png  Matched: Cassiopeia.PNG
Actual: Cassiopeia10.png  Matched: Hercules.PNG
Actual: Cassiopeia11.png  Matched: Gemini.PNG
Actual: Cassiopeia12.png  Matched: Hercules.PNG
Actual: Cassiopeia13.png  Matched: Hercules.PNG
Actual: Cassiopeia14.png  Matched: Hercules.PNG
Actual: Cassiopeia15.png  Matched: Hercules.PNG
Actual: Cassiopeia16.png  Matched: Hercules.PNG
Actual: Cassiopeia17.png  Matched: Hercules.PNG
Actual: Cassiopeia18.png  Matched: Libra.PNG
Actual: Cassiopeia19.png  Matched: Cassiopeia.PNG
Actual: Cassiopeia2.png  Matched: Hercules.PNG
Actual: Cassiopeia3.png  Matched: Gemini.PNG
Actual: Cassiopeia4.png  Matched: Hercules.PNG
Actual: Cassiopeia5.png  Matched: Hercules.PNG
Actual: Cassiopeia6.png  Matched: Hercules.PNG
Actual: Cassiopeia7.png  Matched: Cassiopeia.PNG
Actual: Cassiopeia8.png  Matched: Hercules.PNG
Actual: Cassiopeia9.png  Matched: Hercules.PNG
Actual: Gemini0.png  Matched: Gemini.PNG
Actual: Ge